<a href="https://colab.research.google.com/github/ViacheslavTimofeev/6_project_practicum/blob/main/final_project_7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозирование оттока клиентов в сети отелей «Как в гостях»

### Шаг 1. Откройте файлы с данными

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sys
from google.colab import drive
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier


drive.mount('/content/drive')

np.random.seed(12345)

train_df = pd.read_csv('/content/drive/My Drive/Datasets/hotel_train.csv')
test_df = pd.read_csv('/content/drive/My Drive/Datasets/hotel_test.csv')

#print(train_df.info())
#print(test_df.info())

train_df['arrival_date_year'] = train_df['arrival_date_year'].astype('object')
test_df['arrival_date_year'] = test_df['arrival_date_year'].astype('object')

train_df = train_df.drop(['country', 'arrival_date_week_number', 'arrival_date_day_of_month', 'arrival_date_year'], axis=1)
test_df = test_df.drop(['country', 'arrival_date_week_number', 'arrival_date_day_of_month', 'arrival_date_year'], axis=1)
train_df['reserved_room_type'] = train_df['reserved_room_type'].replace(['A               ', 'B               ', 'C               ', 'D               ', 'E               ', 'F               ', 'G               '], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
test_df['reserved_room_type'] = test_df['reserved_room_type'].replace(['A               ', 'B               ', 'C               ', 'D               ', 'E               ', 'F               ', 'G               '], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
train_df['meal'] = train_df['meal'].replace(['BB       ', 'FB       ', 'HB       ', 'SC       '], ['BB', 'FB', 'HB', 'SC'])
test_df['meal'] = test_df['meal'].replace(['BB       ', 'FB       ', 'HB       ', 'SC       '], ['BB', 'FB', 'HB', 'SC'])


# добавление фич в тестовый датасет, присутствующих в train_df, но отсутствующих в test_df
len_test = len(test_df)

d = {'arrival_date_month_September': np.zeros(len_test),
     'arrival_date_month_October': np.zeros(len_test),
     'arrival_date_month_November': np.zeros(len_test),
     'arrival_date_month_December': np.zeros(len_test)}
add_months = pd.DataFrame(data=d)
test_df = test_df.join(add_months)

# one-hot encoding
df_dummies = pd.get_dummies(train_df)
df_dummies = df_dummies.drop('id', axis=1)
#print(len(df_dummies.columns))

df_dummies_test = pd.get_dummies(test_df)
df_dummies_test = df_dummies_test.drop('id', axis=1)
#print(len(df_dummies_test.columns))

# целевые признаки и фичи
target_train = df_dummies['is_canceled']
features_train = df_dummies.drop('is_canceled', axis=1)

target_test = df_dummies_test['is_canceled']
features_test = df_dummies_test.drop('is_canceled', axis=1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(32412, 25)
(32412, 25)
0        False
1        False
2        False
3        False
4        False
         ...  
32407    False
32408    False
32409    False
32410    False
32411    False
Length: 32412, dtype: bool


### Шаг 2. Предобработка и исследовательский анализ данных

In [ ]:
# выбираем самые влиятельные столбцы для обучения модели
selector = SelectKBest(mutual_info_classif, k=18) # путем изменения k выснилось, что оптимальное значение - 18
features_reduced = selector.fit_transform(features_train, target_train)
cols = selector.get_support(indices=True)

selected_columns = features_train.iloc[:,cols].columns.tolist()
print(selected_columns)

target_train_K_best = df_dummies['is_canceled']
target_test_K_best = df_dummies_test['is_canceled']

#corr_most_col = ['lead_time', 'previous_cancellations', 'booking_changes', 'required_car_parking_spaces', 'total_of_special_requests', 'distribution_channel_Direct', 'distribution_channel_TA/TO']

features_train_K_best = df_dummies[selected_columns]
features_test_K_best = df_dummies_test[selected_columns]

cols_tr_K = features_train_K_best.columns.tolist()
features_test_K_best = features_test_K_best[cols_tr_K]

cols_tr = features_train.columns.tolist()
features_test = features_test[cols_tr]

corr_most_col = ['lead_time',
                 'previous_cancellations',
                 'booking_changes',
                 'required_car_parking_spaces',
                 'total_of_special_requests',
                 'distribution_channel_Direct',
                 'distribution_channel_TA/TO',
                 'days_in_waiting_list',
                 'customer_type_Transient-Party'] # столбцы с cf-коэффициентом корреляции по отношению к целевому признаку. -0.09 >= cf >= 0.09

['lead_time', 'stays_in_week_nights', 'adults', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests', 'total_nights', 'meal_BB', 'distribution_channel_Direct', 'distribution_channel_TA/TO', 'reserved_room_type_A', 'reserved_room_type_D', 'customer_type_Group', 'customer_type_Transient', 'customer_type_Transient-Party']


### Шаг 3. Формулировка ML-задачи на основе бизнес-задачи

Прибыль за 2015-2016: 115293730.0
Прибыль за 2017: 171559330.0


### Шаг 4. Разработка модели ML

In [ ]:
# 1 ТЕСТ: разработка моделей с датафреймом df_dummies (278 столбцов)
  # 1: логистическая регрессия
#model_regression_278 = LogisticRegression(solver='liblinear', class_weight='balanced')
#cross_val_score(model_regression_278, features_train_scaled, target_train, cv=3)
# средний score с балансом классов: 0.59846081
# средний score без баланса классов: 0.6434867933333334

  # 2: random forest
#model_forest_278 = RandomForestClassifier(class_weight='balanced', n_estimators=22, max_depth=10)
#cross_val_score(model_forest_278, features_train_scaled, target_train, cv=3)
# средний score с балансом классов: 0.658
# средний score без баланса классов: 0.7091937633333334

# 2 ТЕСТ: разработка моделей только с самыми показательными столбцами (30 столбцов, выбрано с помощью SelectKBest)
  #1: логистическая регрессия
#model_regression_30 = LogisticRegression(solver='liblinear')
#cross_val_score(model_regression_30, features_train_scaled, target_train, cv=3)
# средний score с балансом классов: 0.59162
# средний score без баланса классов: 0.64432
  # 2: random forest
#model_forest_30 = RandomForestClassifier(n_estimators=100, max_depth=10)
#model_forest_30.fit(features_train_K_best, target_train_K_best)
#cross_val_score(cat, features_train, target_train, cv=3)
# средний score с балансом классов: 0.65973
# средний score без баланса классов: 0.71463
#print(sum([0.6960861 , 0.54633675, 0.67557375])/ 3)
#forest_predict = model_forest_30.predict(features_test_K_best)
#print(accuracy_score(target_test, cat_predict))
# K=20: 0.73, K=18: 0.736, K=25: 0.736


Learning rate set to 0.061334
0:	learn: 0.6499889	total: 59.1ms	remaining: 59s
1:	learn: 0.6152685	total: 108ms	remaining: 53.9s
2:	learn: 0.5896003	total: 154ms	remaining: 51.2s
3:	learn: 0.5680345	total: 201ms	remaining: 50.1s
4:	learn: 0.5514768	total: 245ms	remaining: 48.7s
5:	learn: 0.5366941	total: 288ms	remaining: 47.8s
6:	learn: 0.5235762	total: 331ms	remaining: 46.9s
7:	learn: 0.5118594	total: 373ms	remaining: 46.2s
8:	learn: 0.5029755	total: 410ms	remaining: 45.2s
9:	learn: 0.4929165	total: 455ms	remaining: 45s
10:	learn: 0.4857386	total: 477ms	remaining: 42.9s
11:	learn: 0.4784669	total: 511ms	remaining: 42.1s
12:	learn: 0.4735940	total: 536ms	remaining: 40.7s
13:	learn: 0.4687950	total: 558ms	remaining: 39.3s
14:	learn: 0.4648072	total: 582ms	remaining: 38.2s
15:	learn: 0.4601970	total: 604ms	remaining: 37.2s
16:	learn: 0.4572669	total: 646ms	remaining: 37.4s
17:	learn: 0.4545927	total: 686ms	remaining: 37.4s
18:	learn: 0.4516342	total: 726ms	remaining: 37.5s
19:	learn: 0.4

### Шаг 5. Опишите портрет «ненадёжного» клиента

In [ ]:
forest_predict_series = pd.Series(data=forest_predict)
forest_predict_series = forest_predict_series.rename('is_predicted')

profit = []
loss = []
summer = ['June', 'July', 'August']
autumn = ['September', 'October', 'November']
spring = ['March', 'April', 'May']

def profit_calc(df):  # функция расчета прибыли
    room_prices = {
        'A': {'room': 1000, 'clean': 400},
        'B': {'room': 800, 'clean': 350},
        'C': {'room': 600, 'clean': 350},
        'D': {'room': 550, 'clean': 150},
        'E': {'room': 500, 'clean': 150},
        'F': {'room': 450, 'clean': 150},
        'G': {'room': 350, 'clean': 150}  # словарь категорий комнат
    }

    for _, row in df.iterrows():
        category = row['reserved_room_type']  # присвоение категории каждому элементу датафрейма
        month = row['arrival_date_month']  # проверка на время года
        cf = 1.4 if month in summer else 1.2 if month in spring else 1.2 if month in autumn else 1  # расчет сезонных коэффициентов

        room_price = room_prices[category]['room']
        clean_price = room_prices[category]['clean']
        total_nights = row['total_nights']
        is_canceled = row['is_canceled']
        is_predicted = row['is_predicted']

        if is_canceled == 1 and is_predicted == 1:
            calc_profit = (room_price + clean_price) * cf * 0.8  # расчет прибыли, если бронь отменена и модель предсказала это
            profit.append(calc_profit)
        elif is_canceled == 1 and is_predicted == 0:
            calc_loss =  0 - (room_price * cf + clean_price * cf)  # расчет убытков, если бронь отменена и модель не предсказала это
            loss.append(calc_loss)
        else:
            calc_profit = total_nights * room_price * cf + int(total_nights / 2) * clean_price * cf  # расчет прибыли, если клиент не отменил бронь
            profit.append(calc_profit)

    sum_profit = sum(profit)
    sum_loss = sum(loss)

    return sum_profit, sum_loss
print('Прибыль после внедрения модели:', sum(profit_calc(test_df)))

#Прибыль без модели машинного обучения: 56265600.0
#Прибыль после внедрения модели: 72826986.0
#Чистая прибыль с учетом расходов на разработку модели: 16161386.0



0        1
1        0
2        0
3        0
4        1
        ..
32407    1
32408    0
32409    0
32410    0
32411    1
Name: is_predicted, Length: 32412, dtype: int64
       is_predicted  is_canceled
0                 1            1
1                 0            1
2                 0            1
3                 0            1
4                 1            1
...             ...          ...
32407             1            0
32408             0            0
32409             0            0
32410             0            0
32411             1            0

[32412 rows x 2 columns]
Index(['id', 'is_canceled', 'lead_time', 'arrival_date_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'distribution_channel', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'booking_changes', 'days_in_waiting_list',
       'customer_type', 'required_car_parking_spaces',
  

### Шаг 6. Напишите общий вывод